# 第6章: 単語ベクトル

単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して、以下の処理を行うプログラムを作成せよ。

## 50. 単語ベクトルの読み込みと表示

Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）をダウンロードし、"United States"の単語ベクトルを表示せよ。ただし、"United States"は内部的には"United_States"と表現されていることに注意せよ。

In [2]:
!pip install gensim

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ------- -------------------------------- 4.5/24.0 MB 24.4 MB/s eta 0:00:01
   --------------- ------------------------ 9.4/24.0 MB 23.5 MB/s eta 0:00:01
   ----------------------- ---------------- 13.9/24.0 MB 24.2 MB/s eta 0:00:01
   ------------------------------ --------- 18.1/24.0 MB 21.9 MB/s eta 0:00:01
   ----------------------------------- ---- 21.2/24.0 MB 20.6 MB/s eta 0:00:01
   ---------------------------------------  23.9/24.0 MB 19.4 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 17.1 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   - -------------------------------------- 1.6/46.2 MB 8.4 MB/s eta 0:00:06
   -- ------------------------------------- 3.1/46.2 MB 7.7 MB/s eta 0:00:06
   --- ---------------

In [5]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format(
    "data\GoogleNews-vectors-negative300.bin",
    binary=True,
)

print(model["United_States"])

[-3.61328125e-02 -4.83398438e-02  2.35351562e-01  1.74804688e-01
 -1.46484375e-01 -7.42187500e-02 -1.01562500e-01 -7.71484375e-02
  1.09375000e-01 -5.71289062e-02 -1.48437500e-01 -6.00585938e-02
  1.74804688e-01 -7.71484375e-02  2.58789062e-02 -7.66601562e-02
 -3.80859375e-02  1.35742188e-01  3.75976562e-02 -4.19921875e-02
 -3.56445312e-02  5.34667969e-02  3.68118286e-04 -1.66992188e-01
 -1.17187500e-01  1.41601562e-01 -1.69921875e-01 -6.49414062e-02
 -1.66992188e-01  1.00585938e-01  1.15722656e-01 -2.18750000e-01
 -9.86328125e-02 -2.56347656e-02  1.23046875e-01 -3.54003906e-02
 -1.58203125e-01 -1.60156250e-01  2.94189453e-02  8.15429688e-02
  6.88476562e-02  1.87500000e-01  6.49414062e-02  1.15234375e-01
 -2.27050781e-02  3.32031250e-01 -3.27148438e-02  1.77734375e-01
 -2.08007812e-01  4.54101562e-02 -1.23901367e-02  1.19628906e-01
  7.44628906e-03 -9.03320312e-03  1.14257812e-01  1.69921875e-01
 -2.38281250e-01 -2.79541016e-02 -1.21093750e-01  2.47802734e-02
  7.71484375e-02 -2.81982

## 51. 単語の類似度

"United States"と"U.S."のコサイン類似度を計算せよ。

In [9]:
import torch
import gensim
from torch.nn.functional import cosine_similarity as torch_cosine_similarity

model = gensim.models.KeyedVectors.load_word2vec_format(
    "data\GoogleNews-vectors-negative300.bin", 
    binary=True  
)

vec1 = torch.from_numpy(model["United_States"])
vec2 = torch.from_numpy(model["U.S."])

# 次元を合わせて類似度を計算（1次元Tensor → 2次元バッチ）
similarity = torch_cosine_similarity(vec1.unsqueeze(0), vec2.unsqueeze(0)).item()

print(f"Cosine similarity: {similarity:.4f}")

Cosine similarity: 0.7311


## 52. 類似度の高い単語10件

"United States"とコサイン類似度が高い10語と、その類似度を出力せよ。

In [11]:
from pprint import pprint
import torch
import gensim
from torch.nn.functional import cosine_similarity as torch_cosine_similarity

model = gensim.models.KeyedVectors.load_word2vec_format(
    "data\GoogleNews-vectors-negative300.bin", binary=True
)

similarity_list = model.most_similar(
    positive=["United_States", "U.S."], topn=10
)
pprint(similarity_list)

[('Unites_States', 0.7386658787727356),
 ('Untied_States', 0.6893848180770874),
 ('United_Sates', 0.6722261309623718),
 ('theU.S.', 0.58777916431427),
 ('America', 0.5706374645233154),
 ('Canada', 0.5692834258079529),
 ('Europe', 0.5636841058731079),
 ('IG_TEC_;)_COUNTRY', 0.5610108375549316),
 ('theUnited_States', 0.5585321187973022),
 ('UnitedStates', 0.5390111804008484)]


## 53. 加法構成性によるアナロジー

"Spain"の単語ベクトルから"Madrid"のベクトルを引き、"Athens"のベクトルを足したベクトルを計算し、そのベクトルと類似度の高い10語とその類似度を出力せよ。

## 54. アナロジーデータでの実験

[単語アナロジーの評価データ](http://download.tensorflow.org/data/questions-words.txt)をダウンロードし、国と首都に関する事例（`: capital-common-countries`セクション）に対して、vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し、そのベクトルと類似度が最も高い単語と、その類似度を求めよ。求めた単語と類似度は、各事例と一緒に記録せよ。

## 55. アナロジータスクでの正解率

54の実行結果を用い、意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ。

## 56. WordSimilarity-353での評価

[The WordSimilarity-353 Test Collection](http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.html)の評価データをダウンロードし、単語ベクトルにより計算される類似度のランキングと、人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ。

## 57. k-meansクラスタリング

国名に関する単語ベクトルを抽出し、k-meansクラスタリングをクラスタ数k=5として実行せよ。

## 58. Ward法によるクラスタリング

国名に関する単語ベクトルに対し、Ward法による階層型クラスタリングを実行せよ。さらに、クラスタリング結果をデンドログラムとして可視化せよ。

## 59. t-SNEによる可視化

ベクトル空間上の国名に関する単語ベクトルをt-SNEで可視化せよ。